In [44]:
!pip install imbalanced-learn

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


ERROR: Could not install packages due to an OSError: [WinError 5] Toegang geweigerd: 'C:\\Users\\alexa\\anaconda3\\Lib\\site-packages\\sklearn\\~_check_build\\_check_build.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
import pickle
import nltk
from sklearn.model_selection import KFold,cross_val_score,train_test_split,cross_val_predict
from sklearn import svm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_excel('C:/Users/alexa/OneDrive/Documenten/Thesis Lab/dataset_final.xlsx')

In [3]:
df

,Index,Filename,Countrycode,Label,Sentence_translated,Sentence_original
0,0,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,Law on the Establishment of an Economic Stabil...,'Gesetz zur Errichtung eines Wirtschaftsstabil...
1,1,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,I p. 543 (no.,"'I S. 543 (Nr.',"
2,2,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,"14); Valid from March 28th, 2020 4 changes | P...",'14); Geltung ab 28.03.2020 4 Änderungen | Dr...
3,3,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,28th,"'28.',"
4,4,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,March 2020 StFG § 6- § 8- § 10- § 14e- § 15- §...,'März 2020 StFG § 6- § 8- § 10- § 14e- § 15- ...
...,...,...,...,...,...,...
5445,5445,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Second final provision.,"'Disposición final segunda.',"
5446,5446,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Entry into force.,"'Entrada en vigor.',"
5447,5447,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,This royal decree-law will enter into force on...,'Este real decreto-ley entrará en vigor el dí...
5448,5448,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,"Given in Madrid- on March 10, 2020.","'Dado en Madrid- el 10 de marzo de 2020.',"


# removal of stopwords (in case of multi-lingual model)

In [4]:
stops = set(stopwords.words('spanish'))
stops_french = set(stopwords.words('french'))
stops_italian = set(stopwords.words('italian'))
stops_german = set(stopwords.words('german'))

stops = stops.union(stops_french, stops_italian, stops_german)

In [5]:
df['Sentence_original'] = df['Sentence_original'].apply(word_tokenize)

In [6]:
df['Sentence_original'] = df['Sentence_original'].apply(lambda words: [word.lower() for word in words if word not in stops])

In [7]:
df['Sentence_original'] = df['Sentence_original'].apply(lambda x: ' '.join(map(str, x)))

In [8]:
df

,Index,Filename,Countrycode,Label,Sentence_translated,Sentence_original
0,0,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,Law on the Establishment of an Economic Stabil...,'gesetz errichtung wirtschaftsstabilisierungsf...
1,1,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,I p. 543 (no.,"' i s. 543 ( nr . ' ,"
2,2,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,"14); Valid from March 28th, 2020 4 changes | P...",'14 ) ; geltung ab 28.03.2020 4 änderungen | d...
3,3,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,28th,"'28 . ' ,"
4,4,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,March 2020 StFG § 6- § 8- § 10- § 14e- § 15- §...,'märz 2020 stfg § 6- § 8- § 10- § 14e- § 15- §...
...,...,...,...,...,...,...
5445,5445,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Second final provision.,"'disposición final segunda . ' ,"
5446,5446,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Entry into force.,"'entrada vigor . ' ,"
5447,5447,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,This royal decree-law will enter into force on...,'este real decreto-ley entrará vigor día sigui...
5448,5448,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,"Given in Madrid- on March 10, 2020.","'dado madrid- 10 marzo 2020 . ' ,"


# Majority Baseline

In [4]:
len(df['Label'])

5450

In [23]:
df['Label'].value_counts()

NONE                                             4123
FINANCIAL POLICY: DIRECT PAYMENTS                 387
EMPLOYMENT POLICY                                 254
FINANCIAL POLICY: GUARANTEES                      205
FISCAL POLICY                                     188
SOCIAL WELFARE POLICY                              99
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS      75
CONSUMER PROTECTION                                48
PRICE CONTROL                                      38
INDUSTRIAL POLICY                                  33
Name: Label, dtype: int64

In [5]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(df['Sentence_translated'], df['Label'])

DummyClassifier(strategy='stratified')

In [6]:
dummy_clf.predict(df['Sentence_translated'])

array(['NONE', 'FINANCIAL POLICY: DIRECT PAYMENTS', 'NONE', ..., 'NONE',
       'NONE', 'NONE'], dtype=object)

In [7]:
dummy_clf.score(df['Sentence_translated'], df['Label'])

0.5897247706422019

# Training

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_translated'], df['Label'], test_size=0.2, random_state=111)

In [20]:
vectorizer = FeatureUnion([
    ('tfidf_word', TfidfVectorizer(ngram_range=(1, 2), strip_accents='unicode', lowercase=True, stop_words='english')),
    ('tfidf_word2', TfidfVectorizer(analyzer='word', ngram_range=(1, 5), strip_accents='unicode', lowercase=True, stop_words='english')),
    ('tfidf_char', TfidfVectorizer(analyzer='char', ngram_range=(1, 10), strip_accents='unicode', lowercase=True, stop_words='english'))
])

In [21]:
clf = Pipeline([
('vectorize', vectorizer),
('classify',LinearSVC())
])

In [39]:
tfidf_bow_translated = clf.fit(X_train,y_train)

In [42]:
pickle.dump(tfidf_bow_translated, open('tfidf_bow_translated.pkl', 'wb'))

In [40]:
test_prediction_1 = tfidf_bow_translated.predict(X_test)

In [41]:
print("prediction on trained_model_1 (word trigram + char bi-gram)")
print(classification_report(y_test, test_prediction_1))

prediction on trained_model_1 (word trigram + char bi-gram)
                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       0.83      0.62      0.71         8
                            EMPLOYMENT POLICY       0.82      0.70      0.76        40
            FINANCIAL POLICY: DIRECT PAYMENTS       0.74      0.74      0.74        69
                 FINANCIAL POLICY: GUARANTEES       0.80      0.75      0.78        44
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.89      0.53      0.67        15
                                FISCAL POLICY       0.81      0.70      0.75        30
                            INDUSTRIAL POLICY       0.50      0.20      0.29        10
                                         NONE       0.92      0.96      0.94       841
                                PRICE CONTROL       0.86      0.60      0.71        10
                        SOCIAL WELFARE POLICY       0.80      0.35   

# Cross Validation on tfidfvectorizer

#1 Using English-translated Sentences

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_translated'], df['Label'], test_size=0.2, random_state=0)

In [36]:
from sklearn import metrics
scores_f1 = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')

In [37]:
print(scores_f1)


[0.86009174 0.88302752 0.91743119 0.90366972 0.88302752 0.88302752
 0.88073394 0.87155963 0.88761468 0.91284404]


In [38]:
import statistics
average = statistics.mean(scores_f1)
print(average)

0.888302752293578


#2 Using Multilingual Sentences

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_original'], df['Label'], test_size=0.2, random_state=0)

In [23]:
from sklearn import metrics
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='recall_macro')

In [24]:
print(scores)

[0.53472783 0.66785101 0.57439574 0.52345546 0.64070594]


In [25]:
import statistics
average = statistics.mean(scores)
print(average)

0.5882271974218695
